In [1]:
# import dependencies
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score

import mlflow

In [2]:
mlflow.set_tracking_uri("sqlite:///mydb.sqlite")
#mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///C:/Users/AIDOO K. ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1', creation_time=1703064795646, experiment_id='1', last_update_time=1703064795646, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
# A function for reading the dataset
# Returns a dataframe
def read_dataframe(filename):
    df = pd.read_parquet(filename) 

    df['lpep_dropoff_datetime']= pd.to_datetime(df.lpep_dropoff_datetime)
    df['lpep_pickup_datetime'] = pd.to_datetime(df.lpep_pickup_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    numerical   = ['trip_distance']
    
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
# Receive the dataframe 
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val   = read_dataframe('data/green_tripdata_2021-02.parquet')

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
# Create a new column 'PU_DO'
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
# Performs one-hot encoding of the categorical variables
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
# Extract the target variables
y_train = df_train['duration'].values
y_val   = df_val['duration'].values

In [9]:
# Defines and train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predictions
y_pred = lr.predict(X_train)
val_pred = lr.predict(X_val)

# Train metrics Calculation
train_rmse = mean_squared_error(y_train, y_pred, squared=False)
train_r2   = r2_score(y_train, y_pred)
    
# Validation metrics calculation
val_rmse = mean_squared_error(y_val, val_pred, squared=False)
val_r2   = r2_score(y_val, val_pred)
    

# Metrics printing
print('train mse score:',train_rmse)
print('train r2-score:', train_r2)
    
print('\nval_rmse: ', val_rmse)
print('\nval_r2: ', val_r2)


train mse score: 5.6995641181989765
train r2-score: 0.7570393923885601

val_rmse:  7.758715203341164

val_r2:  0.5937204154722253


In [ ]:
with open('model/linReg.bin', 'wb' ) as file:
    pickle.dump((dv, lr), file)

In [13]:
# Defining mlflow track-keeping
with mlflow.start_run():
    
    mlflow.set_tag("developer", 'Enoch')

    mlflow.log_param("train_data path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid_data path", "data/green_tripdata_2021-02.parquet")
    
    alpha = 0.001
    mlflow.log_param('alpha', alpha)
    
    # with LASSO Regularization
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)

    # Predictions
    y_pred = lr.predict(X_train)
    val_pred = lr.predict(X_val)

    # Train metrics Calculation
    train_rmse = mean_squared_error(y_train, y_pred, squared=False)
    train_r2   = r2_score(y_train, y_pred)
    
    # Validation metrics calculation
    val_rmse = mean_squared_error(y_val, val_pred, squared=False)
    val_r2   = r2_score(y_val, val_pred)
    
    # mlflow tracking
    mlflow.log_metric("train_rmse", train_rmse)
    mlflow.log_metric("train_r2-score", train_r2)
    mlflow.log_metric("valid_rmse", val_rmse)
    mlflow.log_metric("valid-r2", val_r2)


    # Metrics printing
    print('train mse score:',train_rmse)
    print('train r2-score:', train_r2)
    
    print('\nval_rmse: ', val_rmse)
    print('\nval_r2: ', val_r2)

    mlflow.log_artifact(local_path="model/lr1.bin", artifact_path="models_pickle")
    

train mse score: 8.345478908061276
train r2-score: 0.4790991029135001

val_rmse:  9.233436225720547

val_r2:  0.4245970472706959


In [15]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", 'Enoch')

    mlflow.log_param("train_data path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid_data path", "data/green_tripdata_2021-02.parquet")
    
    alpha = 0.001
    mlflow.log_param('alpha', alpha)

    # with RIDGE Regularization
    lr = Ridge(alpha=alpha)
    lr.fit(X_train, y_train)

    # Predictions
    y_pred = lr.predict(X_train)
    val_pred = lr.predict(X_val)

    # Train metrics Calculation
    train_rmse = mean_squared_error(y_train, y_pred, squared=False)
    train_r2   = r2_score(y_train, y_pred)
    
    # Validation metrics calculation
    val_rmse = mean_squared_error(y_val, val_pred, squared=False)
    val_r2   = r2_score(y_val, val_pred)
    
    # mlflow tracking
    mlflow.log_metric("train_rmse", train_rmse)
    mlflow.log_metric("train_r2-score", train_r2)
    mlflow.log_metric("valid_rmse", val_rmse)
    mlflow.log_metric("valid-r2", val_r2)


    # Metrics printing
    print('train mse score:',train_rmse)
    print('train r2-score:', train_r2)
    
    print('\n val_rmse: ', val_rmse)
    print('\n val_r2: ', val_r2)

    mlflow.log_artifact(local_path="model/lr1.bin", artifact_path="models_pickle")

train mse score: 4.944550134760984
train r2-score: 0.817145291823711

 val_rmse:  7.510909603240781

 val_r2:  0.6192582965973402


In [12]:
with open('model/lr1.bin', 'wb' ) as file:
    pickle.dump((dv, lr), file)

In [18]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha' : hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials())
    

[12:03:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:15.07956                                                                                           
[1]	validation-rmse:11.38439                                                                                           
[2]	validation-rmse:9.25740                                                                                            
[3]	validation-rmse:8.05949                                                                                            
[4]	validation-rmse:7.41947                                                                                            
[5]	validation-rmse:7.07521                                                                                            
[6]	validation-rmse:6.87221                      

KeyboardInterrupt: 

In [22]:
 # Supposed these are the best model metrics
params = {
    'learning_rate': 0.20472169880371677,
    'max_depth': 17,
    'min_child_weight': 1.240261172,
    'objective': 'reg:linear',
    'reg_alpha': 0.28567896,
    'reg_lambda': 0.00426448,
    'seed': 42
}

mlflow

mlflow.xgboost.autolog()
booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )

2023/12/20 14:37:53 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bf25fbfa672a4390b35aaa48b1437eb5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[14:38:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.57670
[1]	validation-rmse:14.77950
[2]	validation-rmse:12.64199
[3]	validation-rmse:11.02820
[4]	validation-rmse:9.82436
[5]	validation-rmse:8.93793
[6]	validation-rmse:8.29597
[7]	validation-rmse:7.82922
[8]	validation-rmse:7.49422
[9]	validation-rmse:7.25124
[10]	validation-rmse:7.07616
[11]	validation-rmse:6.94884
[12]	validation-rmse:6.85678
[13]	validation-rmse:6.78789
[14]	validation-rmse:6.73514
[15]	validation-rmse:6.69523
[16]	validation-rmse:6.66330
[17]	validation-rmse:6.64037
[18]	validation-rmse:6.62226
[19]	validation-rmse:6.60733
[20]	validation-rmse:6.59614
[21]	validation-rmse:6.58815
[22]	validation-rmse:6.58247
[23]	validation-rmse:6.57662
[24]	validation-rmse:6.57243
[25]	validation-rmse:6.56791
[26]	validation-rmse:

2023/12/20 14:41:31 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


In [25]:
mlflow.xgboost.autolog(disable=True)
# Model Management

In [26]:
with mlflow.start_run():
    best_params = {
    'learning_rate': 0.20472169880371677,
    'max_depth': 17,
    'min_child_weight': 1.240261172,
    'objective': 'reg:linear',
    'reg_alpha': 0.28567896,
    'reg_lambda': 0.00426448,
    'seed': 42
    }

    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("model/preprocessor.b", 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("model/preprocessor.b", artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


[15:06:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.57670
[1]	validation-rmse:14.77950
[2]	validation-rmse:12.64199
[3]	validation-rmse:11.02820
[4]	validation-rmse:9.82436
[5]	validation-rmse:8.93793
[6]	validation-rmse:8.29597
[7]	validation-rmse:7.82922
[8]	validation-rmse:7.49422
[9]	validation-rmse:7.25124
[10]	validation-rmse:7.07616
[11]	validation-rmse:6.94884
[12]	validation-rmse:6.85678
[13]	validation-rmse:6.78789
[14]	validation-rmse:6.73514
[15]	validation-rmse:6.69523
[16]	validation-rmse:6.66330
[17]	validation-rmse:6.64037
[18]	validation-rmse:6.62226
[19]	validation-rmse:6.60733
[20]	validation-rmse:6.59614
[21]	validation-rmse:6.58815
[22]	validation-rmse:6.58247
[23]	validation-rmse:6.57662
[24]	validation-rmse:6.57243
[25]	validation-rmse:6.56791
[26]	validation-rmse:

C:\Python311\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [27]:
# making pandas dataframe prediction

logged_model = 'runs:/eb7e043a05c24fa88d057919d46c3435/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


[15:14:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [28]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: eb7e043a05c24fa88d057919d46c3435

In [29]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

[15:15:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [30]:
y_pred = xgboost_model.predict(valid)

In [31]:
y_pred

array([15.18813  ,  7.2145815, 13.495876 , ..., 13.663579 ,  6.675244 ,
        8.268747 ], dtype=float32)